Union 2.1 Dataset Analysis
==========================

This data uses the distance modulus, $\mu$, to describe the distance rather than the flux or apparent magnitude. We have
$$
    \mu = m - M~,
$$
where $M$ is given by the dataset as being $M(h=0.7) = -19.3081547178$ and is defined below.

So the dataset gives $\mu(z)$ so we need an extra step.

We will be again fitting the six-parameter $\Lambda$CDM model.

We want to be able to fit $\Omega_k \neq 0$, so we need to make sure that we implement the correct function for the comoving distance,
$$
    R_0 S(\eta) = \frac{cH_0^{-1}}{\sqrt{|\Omega_k|}} \sin\mathrm{n}\left(\sqrt{|\Omega_k|}\int^z_0 \frac{\mathrm{d}z'}{H(z')}\right)~,
$$
with sinn = sin for $\Omega_k < 0$, sinh for $\Omega_k > 0$ and if $\Omega_k = 0$ then this leaves only the constant prefactor and the integral.

$$
    H(a) \equiv \frac{\dot{a}}{a} = H_0 \sqrt{ \left [ (\Omega_c + \Omega_b) a^{-3} + \Omega_{rad} a^{-4} + \Omega_k a^{-2} + \Omega_{DE} a^{-3(1+w)} \right ]}~. ~ ~ ~ ~ (1)
$$

In [196]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg', 'retina'}

In [197]:
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import lmfit
from copy import deepcopy
josh_colors = ['#7E317B', '#164B44', '#45CAB9', '#8B7C44']

In [198]:
hubble_constant = 70 * 3.24078e-20
speed_of_light = 3e8

In [199]:
params = lmfit.Parameters()

params.add('lpeak', value=4e39, vary=True)
#omegas
params.add('omega', value=1., vary=False)
params.add('c', value=0., vary=False, min=0.)  # degenerate with b
params.add('b', value=0.3, vary=True, min=0.)
params.add('rad', value=0., vary=True, min=0.)

params.add('de', value=0.7, vary=True, min=0., max=1.)
params.add('k', value=0., vary=True, expr="omega-c-b-rad-de")  # fixed at 0
params.add('w', value=-1, vary=False)  # don't move me man

In [200]:
def friedmann(z, params):
    model = (params['c'].value + params['b'].value) * ((1+z)**3) + \
            params['rad'].value * ((1+z)**4) + \
            params['k'].value * ((1+z)**2) + \
            params['de'].value * ((1+z)**(3*(1 + params['w'].value)))
    return np.sqrt(model)

In [201]:
def sk_integrand(z, params):
    return 1/friedmann(z, params)

In [202]:
def sk_integral(z, params):
    oarray = np.empty_like(z)
    
    for i in range(len(z)):
        output = si.quad(sk_integrand, 0, z[i], args=(params))
        oarray[i] = output[0]
        
    return oarray

In [203]:
def comoving(z, params):
    omegak = params['k'].value
    this_int = sk_integral(z, params)
    k_precision = 1E-9
    
    if -k_precision < omegak < k_precision:
        factor = this_int
    elif omegak < -k_precision:
        factor = (1/np.sqrt(omegak))*np.sin((np.sqrt(omegak))*this_int)
    elif omegak > k_precision:  # omegak > 0 and other edge cases, doesn't really matter
        factor = (1/np.sqrt(omegak))*np.sinh((np.sqrt(omegak))*this_int)
        
    return factor * (speed_of_light/hubble_constant)

In [204]:
def read_data(fname="data/union_21_data.csv"):
    """Reads data

    Output
    ------
    z: z array
    m: magnitude array
    m_err: magnitude error array"""
    
    raw = np.loadtxt(fname, delimiter="\t", usecols=(1,2,3), skiprows=1)
    return raw[:,0], raw[:,1] - 19.3, raw[:,2]

In [205]:
def mag_of_f(f, m0=-20.45):
    """Implementation of (3) at the top. Returns magnitude as a function of flux
    Assumes f given in cgs units"""
    return m0 - 2.5*np.log10(1E-4*f)  # factor to deal with units

In [206]:
def f_of_z(z, params):
    return params['lpeak'].value/(4*np.pi*(comoving(z, params)*(1+z))**2)

In [207]:
def chisq(obs, exp, err):
    """Returns the chi squared as in (5) top writing."""
    return ((obs-exp)**2/(err**2))

In [ ]:
def for_fitting(params, z, data, errors):
    return chisq(mag_of_f(f_of_z(z, params)), data, errors)

In [ ]:
z, mag, mag_err = read_data()
out = lmfit.minimize(for_fitting, params, args=(z, mag, mag_err), method='leastsq')

print(for_fitting(params, z, mag, mag_err).sum()/(580-3))
print(lmfit.fit_report(out))

In [ ]:
plt.errorbar(z, mag, yerr=mag_err, linestyle='none', color=josh_colors[0], marker='o', ms=4, alpha=0.7, capthick=0)

z_for_plot = np.arange(min(z), max(z), 0.01)

m_for_plot = mag_of_f(f_of_z(z_for_plot, params))
params_1_0 = deepcopy(params)
params_1_0['b'].value = 1
params_1_0['de'].value = 0
m_for_plot_1_0 =  mag_of_f(f_of_z(z_for_plot, params_1_0))
params_0_1 = deepcopy(params)
params_0_1['b'].value = 0
params_0_1['de'].value = 1
m_for_plot_0_1 =  mag_of_f(f_of_z(z_for_plot, params_0_1))

plt.plot(z_for_plot, m_for_plot, color=josh_colors[1])
plt.plot(z_for_plot, m_for_plot_1_0, linestyle='--', color=josh_colors[2])
plt.plot(z_for_plot, m_for_plot_0_1, linestyle='-.', color=josh_colors[3])
plt.xlabel("$z$")
plt.ylabel("$m$")
plt.show()